## Model: Responsible Business Simulator (RBS)
In this notebook the core (calculation) steps of the RBS model are illustrated.

### (I): Read data
Firstly, we are going to read in the data. To do this we use the **import_case()** function that is part of the **core**-folder. This function takes two arguments: a _file_format_ (csv, json or xlsx) and a _path_ to the folder containing the data-folders. Note that with folder there should a folder with a name that matches the _file_format_.  

In [2]:
from pathlib import Path
from core.import_case import import_case

file_format = 'xlsx'
path = Path.cwd() / 'data' / 'beerwiser'
import_case(file_format, path)

/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx
/Users/tdijk004/Documents/tRBS/open source/tRBS_model/model/data/beerwiser/xlsx/beerwiser.xlsx


{'configurations': array(['use_theme_weights'], dtype=object),
 'configuration_value': array([0]),
 'key_outputs': array(['Accidents reduction', 'Water use reduction',
        'Production cost reduction'], dtype=object),
 'key_output_unit': array(['#/year', 'hl/year', '%'], dtype=object),
 'key_output_theme': array(['People', 'Planet', 'Profit'], dtype=object),
 'themes': array(['People', 'Planet', 'Profit'], dtype='<U32'),
 'key_output_minimum': array([nan, nan, nan]),
 'key_output_maximum': array([nan, nan, nan]),
 'key_output_monetary': array([0, 0, 0]),
 'key_output_smaller_the_better': array([0, 0, 0]),
 'key_output_linear': array([1, 1, 0]),
 'key_output_automatic': array([1, 1, 1]),
 'key_output_start': array([nan, nan, nan]),
 'key_output_end': array([nan, nan, nan]),
 'key_output_threshold': array([nan, nan, nan]),
 'decision_makers_options': array(['Focus on water recycling', 'Focus on training', 'Equal spread'],
       dtype='<U32'),
 'internal_variable_inputs': array(['Inve

PosixPath('model/data/beerwiser/hoi')